In [7]:
import pandas as pd
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn.metrics import precision_recall_fscore_support as prfs
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.cross_validation import cross_val_score
from sklearn import linear_model
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier


In [9]:
df = pd.read_csv('extrapolated_trained_data_new_1.csv')

In [11]:
df['interest_level'] = df['interest_level'].astype('category')
df['interest_level_codes'] = df['interest_level'].cat.codes

df['display_address'] = df['display_address'].astype('category')
df['display_address_codes'] = df['display_address'].cat.codes

df['latitude'] = df['latitude'].astype('category')
df['latitude_codes'] = df['latitude'].cat.codes

df['longitude'] = df['longitude'].astype('category')
df['longitude_codes'] = df['longitude'].cat.codes

df.head(1)

,bathrooms,bedrooms,building_id,created,description,display_address,interest_level,latitude,listing_id,longitude,...,high_rise,no_fee,parking_space,renovated_apt,midrise,pet_upon_approval,interest_level_codes,display_address_codes,latitude_codes,longitude_codes
0,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,6/24/2016 7:54,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,medium,40.7145,7211212,-73.9425,...,0,0,0,0,0,0,2,4636,793,839


In [13]:
featuresDF = df.drop('interest_level', axis=1)
featuresDF = featuresDF.drop('building_id', axis=1)
featuresDF = featuresDF.drop('created', axis=1)
featuresDF = featuresDF.drop('description', axis=1)
featuresDF = featuresDF.drop('display_address', axis=1)
featuresDF = featuresDF.drop('latitude', axis=1)
featuresDF = featuresDF.drop('longitude', axis=1)
featuresDF = featuresDF.drop('manager_id', axis=1)
featuresDF = featuresDF.drop('photos', axis=1)
featuresDF = featuresDF.drop('street_address', axis=1)
featuresDF = featuresDF.drop('listing_id', axis=1)

In [14]:
y_label = featuresDF['interest_level_codes']
y_label.value_counts()

1    32216
2     9952
0     3174
Name: interest_level_codes, dtype: int64

In [15]:
target_names = ['0', '1', '2'];
X_train, X_test, y_train, y_test = train_test_split(featuresDF,y_label, test_size=0.25,random_state=15,stratify=y_label)
print len(X_train)
print len(X_test)
print len(y_train)
print len(y_test)

34006
11336
34006
11336


In [16]:
print y_train.value_counts();
print y_test.value_counts();

1    24162
2     7464
0     2380
Name: interest_level_codes, dtype: int64
1    8054
2    2488
0     794
Name: interest_level_codes, dtype: int64


In [17]:
def generate_rf(X_train, y_train, X_test, y_test):
    rf = RandomForestClassifier(n_estimators=50, min_samples_leaf=10)
    rf.fit(X_train, y_train)
    print "rf score ", rf.score(X_test, y_test)
    return rf

def combine_rfs(rf_a, rf_b):
    rf_a.estimators_ += rf_b.estimators_
    rf_a.n_estimators = len(rf_a.estimators_)
    return rf_a

In [18]:
rfs = [generate_rf(X_train, y_train, X_test, y_test) for i in xrange(10)]
rf_combined = reduce(combine_rfs, rfs)

print "rf combined score", rf_combined.score(X_test, y_test)

rf score  0.929693013409
rf score  0.923517995766
rf score  0.928634438956
rf score  0.929516584333
rf score  0.929957657022
rf score  0.927487649965
rf score  0.927399435427
rf score  0.935338743825
rf score  0.928546224418
rf score  0.927311220889
rf combined score 0.929075511644


In [19]:
predicted_labels = rf_combined.predict(X_test)

In [20]:
actual_class = y_test.tolist()
predicted_class = predicted_labels.tolist()

In [21]:
target_names = ['0', '1', '2']

In [22]:
print(classification_report(actual_class, predicted_class, target_names=target_names))

             precision    recall  f1-score   support

          0       1.00      0.01      0.01       794
          1       0.91      1.00      0.95      8054
          2       1.00      1.00      1.00      2488

avg / total       0.94      0.93      0.90     11336



In [14]:
#predicted_prob_labels = multinomialNBClassifier.predict_proba(X_test);

In [23]:
score = rf_combined.score(X_test,actual_class);
print score;

0.929075511644


In [ ]:
cvScores = cross_val_score(rf_combined, featuresDF, y_label, cv=10, scoring='f1_micro');
print cvScores;

In [17]:
#print len(predicted_prob_labels)
#prob_labels_list = predicted_prob_labels.tolist()
#print prob_labels_list